In [1]:
# AXILiteS
# 0x000 : Control signals
#         bit 0  - ap_start (Read/Write/COH)
#         bit 1  - ap_done (Read/COR)
#         bit 2  - ap_idle (Read)
#         bit 3  - ap_ready (Read)
#         bit 7  - auto_restart (Read/Write)
#         others - reserved
# 0x004 : Global Interrupt Enable Register
#         bit 0  - Global Interrupt Enable (Read/Write)
#         others - reserved
# 0x008 : IP Interrupt Enable Register (Read/Write)
#         bit 0  - Channel 0 (ap_done)
#         bit 1  - Channel 1 (ap_ready)
#         others - reserved
# 0x00c : IP Interrupt Status Register (Read/TOW)
#         bit 0  - Channel 0 (ap_done)
#         bit 1  - Channel 1 (ap_ready)
#         others - reserved
# 0x100 ~
# 0x1ff : Memory 'symbol_bits_V' (256 * 5b)
#         Word n : bit [ 4: 0] - symbol_bits_V[4n]
#                  bit [12: 8] - symbol_bits_V[4n+1]
#                  bit [20:16] - symbol_bits_V[4n+2]
#                  bit [28:24] - symbol_bits_V[4n+3]
#                  others      - reserved
# 0x200 ~
# 0x2ff : Memory 'codeword_length_histogram_V' (64 * 32b)
#         Word n : bit [31:0] - codeword_length_histogram_V[n]
# 0x400 ~
# 0x7ff : Memory 'encoding_V' (256 * 32b)
#         Word n : bit [31:0] - encoding_V[n]
# (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)

from pynq import Overlay

overlay = Overlay("../bitstream/create-code-word-test.bit")
overlay.download()

create_codeword = overlay.create_codeword_0

# write symbol bits
create_codeword.write(0x100 + 4*24, 50594304)
create_codeword.write(0x100 + 4*25, 262658  )

# create code word len histogram
# code_word_len_histo = [
#     { 'symbol': 'a', 'len': 2 },
#     { 'symbol': 'd', 'len': 2 },
#     { 'symbol': 'e', 'len': 2 },
#     { 'symbol': 'c', 'len': 3 },
#     { 'symbol': 'b', 'len': 4 },
#     { 'symbol': 'f', 'len': 4 },
# ]

symbols = [ 'a', 'b', 'c', 'd', 'e', 'f' ]
code_word_len_histo = [ 0, 0, 3, 1, 2 ]

# code_word_len_histo = [
#     { 'symbol': 'a', 'len': 2 },
#     { 'symbol': 'b', 'len': 4 },
#     { 'symbol': 'c', 'len': 3 },
#     { 'symbol': 'd', 'len': 2 },
#     { 'symbol': 'e', 'len': 2 },
#     { 'symbol': 'f', 'len': 4 },
# ]

for idx, num in enumerate(code_word_len_histo):
    create_codeword.write(0x200 + 4*idx, num)
    
# start
create_codeword.write(0x000, 1)

from time import sleep

sleep(1)

get_bin = lambda x, n: format(x, 'b').zfill(n)

for sym in symbols:
    encoding = create_codeword.read(0x400 + 4*ord(sym))
    print('symbol : ' + sym + ', code word : ' + get_bin(encoding, 32))

symbol : a, code word : 00000000000000000000000000000010
symbol : b, code word : 00000000000000000000000011100100
symbol : c, code word : 00000000000000000000000001100011
symbol : d, code word : 00000000000000000000000001000010
symbol : e, code word : 00000000000000000000000000100010
symbol : f, code word : 00000000000000000000000111100100
